THis is for esp dummy data all features

In [8]:
import numpy as np
import tensorflow as tf
from scipy.stats import kurtosis, skew
from scipy.fft import rfft, rfftfreq

# -----------------------------
# PARAMETERS
# -----------------------------
WINDOW_SIZE = 64
FS = 100.0          # Sampling frequency (Hz)
NUM_SAMPLES = 3000
NUM_CLASSES = 7



In [9]:
# Accelerometer (g)
ax = np.random.normal(0, 0.02, NUM_SAMPLES)
ay = np.random.normal(0, 0.02, NUM_SAMPLES)
az = np.random.normal(1.0, 0.02, NUM_SAMPLES)

# Gyroscope (deg/s)
gx = np.random.normal(0, 1.5, NUM_SAMPLES)
gy = np.random.normal(0, 1.5, NUM_SAMPLES)
gz = np.random.normal(0, 1.5, NUM_SAMPLES)

# RPM
rpm = np.random.normal(1800, 30, NUM_SAMPLES)

# Electrical
current = np.random.normal(0.45, 0.05, NUM_SAMPLES)
temperature = np.random.normal(40, 1.0, NUM_SAMPLES)


In [10]:
def time_features(x):
    rms = np.sqrt(np.mean(x**2))
    p2p = np.ptp(x)
    std = np.std(x)
    krt = kurtosis(x)
    skw = skew(x)
    crest = np.max(np.abs(x)) / (rms + 1e-6)
    return [rms, p2p, std, krt, skw, crest]


def freq_features(x):
    xf = np.abs(rfft(x))
    freqs = rfftfreq(len(x), 1/FS)

    dom_freq = freqs[np.argmax(xf)]
    hf_energy = np.sum(xf[freqs > 500])

    return [dom_freq, hf_energy]


features = []
labels = []

for i in range(NUM_SAMPLES - WINDOW_SIZE):
    win = slice(i, i + WINDOW_SIZE)

    feat = []

    # Accel
    for axis in [ax, ay, az]:
        feat += time_features(axis[win])
        feat += freq_features(axis[win])

    # Gyro
    for axis in [gx, gy, gz]:
        feat += time_features(axis[win])

    # Cross-domain
    vib_rms = np.sqrt(np.mean(ax[win]**2 + ay[win]**2 + az[win]**2))
    feat.append(vib_rms / (np.mean(rpm[win]) + 1e-6))        # Vibration/RPM

    feat.append(np.corrcoef(current[win], temperature[win])[0, 1])  # I/T correlation
    feat.append(np.mean(current[win]) * np.mean(rpm[win]))           # Power proxy

    # Thermal
    feat.append(np.mean(temperature[win]))

    features.append(feat)

    # 🔹 TEMPORARY synthetic labels
    labels.append(np.random.randint(0, NUM_CLASSES))

X = np.array(features, dtype=np.float32)
y = tf.keras.utils.to_categorical(labels, NUM_CLASSES)



In [11]:
mean = X.mean(axis=0)
std  = X.std(axis=0) + 1e-6

X = (X - mean) / std

np.save("scaler_mean.npy", mean)
np.save("scaler_std.npy", std)


In [14]:
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(X.shape[1],)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(NUM_CLASSES, activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)


In [15]:
model.fit(X, y, epochs=25, batch_size=32, verbose=1)

Epoch 1/25
92/92 [==============================] - 1s 2ms/step - loss: 2.0157 - accuracy: 0.1383
Epoch 2/25
92/92 [==============================] - 0s 2ms/step - loss: 1.9542 - accuracy: 0.1649
Epoch 3/25
92/92 [==============================] - 0s 2ms/step - loss: 1.9369 - accuracy: 0.1809
Epoch 4/25
92/92 [==============================] - 0s 2ms/step - loss: 1.9248 - accuracy: 0.1866
Epoch 5/25
92/92 [==============================] - 0s 2ms/step - loss: 1.9171 - accuracy: 0.2006
Epoch 6/25
92/92 [==============================] - 0s 2ms/step - loss: 1.9106 - accuracy: 0.2037
Epoch 7/25
92/92 [==============================] - 0s 2ms/step - loss: 1.9043 - accuracy: 0.2166
Epoch 8/25
92/92 [==============================] - 0s 2ms/step - loss: 1.8994 - accuracy: 0.2173
Epoch 9/25
92/92 [==============================] - 0s 2ms/step - loss: 1.8950 - accuracy: 0.2197
Epoch 10/25
92/92 [==============================] - 0s 2ms/step - loss: 1.8908 - accuracy: 0.2224
Epoch 11/25
92/92 [

In [16]:
def representative_dataset():
    for i in range(300):
        yield [X[i:i+1]]

converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8
converter.inference_output_type = tf.int8

tflite_model = converter.convert()


INFO:tensorflow:Assets written to: /tmp/tmpl0s3b5h4/assets


INFO:tensorflow:Assets written to: /tmp/tmpl0s3b5h4/assets
/home/mariam/NTI/analysis/myenv/lib/python3.8/site-packages/tensorflow/lite/python/convert.py:887: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(
2025-12-30 14:33:30.725632: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2025-12-30 14:33:30.725676: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2025-12-30 14:33:30.725935: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /tmp/tmpl0s3b5h4
2025-12-30 14:33:30.727506: I tensorflow/cc/saved_model/reader.cc:91] Reading meta graph with tags { serve }
2025-12-30 14:33:30.727548: I tensorflow/cc/saved_model/reader.cc:132] Reading SavedModel debug info (if present) from: /tmp/tmpl0s3b5h4
2025-12-30 14:33:30.731366: I tensorflow/cc/saved_model/loader.cc:231] Restoring SavedModel bundle.
2025-12-30 

In [17]:
with open("fault_model.h", "w") as f:
    f.write("const unsigned char fault_model[] = {")
    f.write(",".join(str(b) for b in tflite_model))
    f.write("};\n")
    f.write(f"const unsigned int fault_model_len = {len(tflite_model)};\n")


In [18]:
#conversion

In [19]:
import numpy as np

mean = np.load("scaler_mean.npy")
std  = np.load("scaler_std.npy")

assert mean.shape == std.shape

NUM_FEATURES = mean.shape[0]

with open("scaler_params.h", "w") as f:
    f.write("#ifndef SCALER_PARAMS_H\n")
    f.write("#define SCALER_PARAMS_H\n\n")

    f.write(f"#define NUM_FEATURES {NUM_FEATURES}\n\n")

    # Mean
    f.write("const float scaler_mean[NUM_FEATURES] = {\n")
    for v in mean:
        f.write(f"  {v:.8f}f,\n")
    f.write("};\n\n")

    # Std
    f.write("const float scaler_std[NUM_FEATURES] = {\n")
    for v in std:
        f.write(f"  {v:.8f}f,\n")
    f.write("};\n\n")

    f.write("#endif // SCALER_PARAMS_H\n")
